In [88]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# 모델 로드 (한국어 SBERT)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# CSV 로드
df = pd.read_csv("../data/school_info/dmu_menu_urls.csv")
titles = df['title'].tolist()
urls = df['url'].tolist()

# 타이틀 임베딩
title_vectors = model.encode(titles, convert_to_tensor=True)

In [89]:
from sklearn.metrics.pairwise import cosine_similarity
import torch

def find_best_url(query, top_k=1):
    query_vec = model.encode([query], convert_to_tensor=True)
    scores = cosine_similarity(query_vec.cpu().numpy(), title_vectors.cpu().numpy())[0]
    top_idx = scores.argsort()[-top_k:][::-1]
    results = [(titles[i], urls[i], scores[i]) for i in top_idx]
    return results

In [90]:
query = "경영학부 사이트 알려줘"
result = find_best_url(query)[0]

print(f"📌 관련 메뉴: {result[0]}")
print(f"👉 링크: {result[1]}")

📌 관련 메뉴: 대학생활/경영학부
👉 링크: https://www.dongyang.ac.kr/dmu/4873/subview.do


In [91]:
from sentence_transformers import SentenceTransformer
import pandas as pd

df = pd.read_csv("../data/school_info/dmu_menu_with_text.csv")
cols = ["title", "url", "content"]
df = df.dropna(subset=["content"])  # 본문 없는 경우 제외
df["combined_text"] = df["title"].fillna("") + "\n" + df["content"].fillna("")


model = SentenceTransformer("jhgan/ko-sroberta-multitask")  # 호환 안정 모델

combined_embeddings = model.encode(df["combined_text"].tolist(), convert_to_tensor=True)


In [92]:
from sklearn.metrics.pairwise import cosine_similarity
import torch

def search_url(query, top_k=3):
    q_emb = model.encode([query], convert_to_tensor=True)
    scores = cosine_similarity(q_emb.cpu().numpy(), combined_embeddings.cpu().numpy())[0]
    top_idx = scores.argsort()[-top_k:][::-1]
    return [(df.iloc[i]["title"], df.iloc[i]["url"], scores[i]) for i in top_idx]

for title, url, score in search_url("경영학부주임연락처", top_k=3):
    print(f"✅ {title} ({score:.4f})\n👉 {url}\n")

✅ 학부ㆍ학과/경영학과 (0.7246)
👉 https://www.dongyang.ac.kr/dmu/4670/subview.do

✅ 학부ㆍ학과/경영정보학과 (0.6771)
👉 https://www.dongyang.ac.kr/dmu/4712/subview.do

✅ 학부ㆍ학과/학부소개 (0.6608)
👉 https://www.dongyang.ac.kr/dmu/4669/subview.do



In [93]:
import re

# 데이터 불러오기
df = pd.read_csv("../data/school_info/dmu_menu_with_text.csv")

# title과 text를 결합해서 문맥을 풍부하게 만듦
df['fulltext'] = df['title'] + " " + df['content']

# 문장 임베딩
content_embeddings = model.encode(df['fulltext'].tolist(), convert_to_tensor=True)

from sklearn.metrics.pairwise import cosine_similarity

def find_best_answer(query, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=True)
    scores = cosine_similarity(query_embedding.cpu().numpy(), content_embeddings.cpu().numpy())[0]
    top_idx = scores.argsort()[-top_k:][::-1]
    return df.iloc[top_idx]

def extract_contact_info(text):
    tel = re.search(r'(?:TEL\.?|전화(?:번호)?)[^\d]*(\d{2,4}[-\s]?\d{3,4}[-\s]?\d{4})', text)
    email = re.search(r'(?:E[-]?mail|이메일)[^\w]*([\w\.-]+@[\w\.-]+)', text)
    return {
        '전화번호': tel.group(1) if tel else '없음',
        '이메일': email.group(1) if email else '없음'
    }

In [95]:
query = "학생성공지원"
results = find_best_answer(query)

for _, row in results.iterrows():
    contact = extract_contact_info(row['fulltext'])
    print(f"✅ 제목: {row['title']}")
    print(f"📞 전화번호: {contact['전화번호']}")
    print(f"📧 이메일: {contact['이메일']}")
    print(f"👉 링크: {row['url']}\n")

✅ 제목: 대학소개/학생활동지원팀
📞 전화번호: 02-2618-2300
📧 이메일: 없음
👉 링크: https://www.dongyang.ac.kr/dmu/4375/subview.do

✅ 제목: 대학소개/학생처
📞 전화번호: 02-2618-2300
📧 이메일: 없음
👉 링크: https://www.dongyang.ac.kr/dmu/4375/subview.do

✅ 제목: 대학소개/교육지원팀
📞 전화번호: 02-2610-1711
📧 이메일: 없음
👉 링크: https://www.dongyang.ac.kr/dmu/4365/subview.do

